---
<h1 align="center"><strong>Cálculo de dependencia cruzada para todos los países</strong></h1>
<h4 align="center"><strong>Manuel Alejandro Hidalgo y Jorge Díaz Lanchas</strong></h4>
<h4 align="center"><strong>Fundación Real Instituto Elcano</strong></h4>

---

# Esquema para Índice de Seguridad Económica - Real Instituto Elcano

## 1. Introducción y Marco Conceptual

- **Objetivo**: Desarrollar un índice que cuantifique la seguridad económica de los países
- **Definición**: La seguridad económica como la capacidad de un país para resistir disrupciones en sus cadenas de suministro y comercio internacional
- **Relevancia**: Contexto actual de fragmentación geoeconómica y tensiones comerciales

## 2. Metodología

### 2.1 Fuentes de Datos
- Base de datos International Trade and Production Database (ITP)
- Datos comerciales bilaterales por industria (año 2019)
- Otros indicadores macroeconómicos complementarios

### 2.2 Procesamiento de Datos
```python
# Procesamiento y carga de datos ITP
itp2019, codigos_countries = procesar_datos_itp()
```

### 2.3 Creación de Matrices de Comercio
```python
# Generación de matrices bilaterales por industria
matrices_comercio = crear_matriz_comercio(data.groupby('industry_descr'), codigos_paises)
```

### 2.4 Limpieza y Filtrado
```python
# Eliminar relaciones comerciales insignificantes
mat_clean = eliminar_filas_columnas_cero(mat, threshold_pct=0.05)
```

### 2.5 Cálculo de Dependencias Económicas
```python
# Cálculo de dependencias directas e indirectas
results = analyze_dependencies(X, country_names)
```

## 3. Componentes del Índice

### 3.1 Dependencia Directa
- Medición de la dependencia inmediata entre países
- Fórmula: $D_{ij} = \frac{X_{ji}}{∑_k X_{ki}}$, donde $X_{ji}$ es el comercio del país j al país i

### 3.2 Dependencia Indirecta
- Medición de dependencias a través de cadenas de suministro
- Incorporación de países intermediarios en las relaciones comerciales
- Análisis de caminos hasta longitud 5

In [1]:
import os
import gzip
import pandas as pd
import numpy as np
from numpy.linalg import inv
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import combinations
import torch
import dask.dataframe as dd
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict
import multiprocessing
from joblib import Parallel, delayed

***No ejecutar este código a menos que se quiera comprimir***

In [12]:
def comprimir_dividir_archivo(archivo_original, tamano_maximo=100, directorio_salida=None):
    # Asegúrate de que el archivo original existe
    archivo_original = Path(archivo_original)
    if not archivo_original.exists():
        raise FileNotFoundError(f"No se encuentra el archivo: {archivo_original}")
    
    # Si no se especifica directorio de salida, usar src/data/raw/ITP/
    if directorio_salida is None:
        # Obtener el directorio raíz del proyecto (donde está src/)
        proyecto_root = Path.cwd().parent.parent
        directorio_salida = proyecto_root /'data' / 'raw' / 'ITP' / 'ITPD_E_R03'
    else:
        directorio_salida = Path(directorio_salida)
    
    # Crear el directorio de salida si no existe
    directorio_salida.mkdir(parents=True, exist_ok=True)
    
    # Abre el archivo original en modo de lectura binaria
    with open(archivo_original, 'rb') as f_in:
        # Lee el contenido del archivo original
        contenido = f_in.read()
        
        # Determina el número de partes necesarias
        num_partes = (len(contenido) + tamano_maximo - 1) // tamano_maximo
        
        # Divide el contenido en partes y escribe cada parte comprimida
        for i in range(num_partes):
            parte = contenido[i * tamano_maximo: (i + 1) * tamano_maximo]
            archivo_salida = directorio_salida / f'ITPD_E_R03.csv.parte{i}.gz'
            with gzip.open(archivo_salida, 'wb') as f_out:
                f_out.write(parte)
            print(f"Parte {i} creada en: {archivo_salida}")

# Tamaño máximo por parte (1GB)
tamano_maximo = 810 * 1024 * 1024

try:
    # Ruta al archivo original
    archivo_original = Path(r"C:\Users\Usuario\Downloads\ITPDE_R03\ITPDE_R03.csv")
    
    # Comprimir y dividir el archivo original
    comprimir_dividir_archivo(archivo_original, tamano_maximo)
    print("Proceso completado con éxito")
except Exception as e:
    print(f"Error durante el proceso: {e}")

Parte 0 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.parte0.gz
Parte 1 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.parte1.gz
Parte 2 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.parte2.gz
Parte 3 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.parte3.gz
Parte 4 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.parte4.gz
Parte 5 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.parte5.gz
Parte 6 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.parte6.gz
Parte 7 creada en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\raw\ITP\ITPD_E_R03\ITPD_E_R03.csv.pa

### ***Descomprimir, carga de datos y borrado de archivo***

La compresión se hace para poder trabajar con git sin porblemas de tamaño de ficheros.
Se descomprime, se importa y luego se borra el fichero descomprimido


In [2]:
"""
FASE 1: PREPARACIÓN Y CARGA DE DATOS
Este script procesa la base de datos International Trade and Production Database (ITP)
que viene dividida en múltiples archivos comprimidos, utilizando aceleración GPU
cuando está disponible.
"""


def procesar_datos_itp():
   try:
       # Verificar si GPU está disponible
       gpu_disponible = torch.cuda.is_available()
       if gpu_disponible:
           print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
       else:
           print("GPU no disponible, se usará CPU")
       
       # Definición de rutas usando Path y la estructura de tu proyecto
       try:
           base_path = Path(__file__).parent
       except NameError:  # Estamos en un notebook
           base_path = Path.cwd().parent.parent  # Asumiendo que el notebook está en /notebooks/

       source_directory = base_path / "data" / "raw" / "ITP" / "ITPD_E_R03"
       target_directory = base_path / "data" / "processed"
       target_filename = 'ITPD_E_R03.csv'

       # Imprimir las rutas para verificación
       print(f"Directorio fuente: {source_directory}")
       print(f"Directorio destino: {target_directory}")

       # Asegurar que los directorios existen
       target_directory.mkdir(parents=True, exist_ok=True)

       # Verificar que el directorio fuente existe
       if not source_directory.exists():
           raise FileNotFoundError(f"No se encuentra el directorio fuente: {source_directory}")

       # Listar archivos comprimidos
       chunk_filenames = sorted([f for f in os.listdir(source_directory) 
                        if f.startswith('ITPD_E_R03.csv.parte') and f.endswith('.gz')])

       # Control de errores: verificar que existen archivos para procesar
       if not chunk_filenames:
           raise FileNotFoundError(f"No se encontraron archivos .gz en {source_directory}")

       # Construir la ruta completa para el archivo combinado
       target_filepath = target_directory / target_filename

       # Función para descomprimir un archivo en paralelo
       def descomprimir_archivo(chunk_filename):
           chunk_filepath = source_directory / chunk_filename
           with gzip.open(chunk_filepath, 'rb') as chunk_file:
               return chunk_file.read()

       print("Combinando archivos comprimidos...")
       with open(target_filepath, 'wb') as target_file:
           # Usar ThreadPoolExecutor para paralelizar la descompresión
           with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
               for data in tqdm(
                   executor.map(descomprimir_archivo, chunk_filenames),
                   total=len(chunk_filenames),
                   desc="Procesando archivos"
               ):
                   target_file.write(data)

       print("Leyendo archivo CSV...")
       
       # Usar Dask para procesamiento paralelo
       print("Usando Dask para procesamiento en paralelo")
       dask_df = dd.read_csv(target_filepath, sep=",")
       
       # Filtrar año 2022 eficientemente
       print("Filtrando datos de 2022...")
       itp2022 = dask_df[dask_df['year'] == 2022].compute()
       
       # Limpieza: eliminar archivo temporal
       os.remove(target_filepath)
       print(f"Archivo temporal eliminado")

       # Obtener lista única de países importadores - usando GPU si disponible
       if gpu_disponible:
           # Transferir a GPU para operaciones de unique más rápidas
           importer_tensor = torch.tensor(
               pd.factorize(itp2022['importer_iso3'])[0], 
               device='cuda'
           )
           unique_indices = torch.unique(importer_tensor).cpu().numpy()
           # Mapear índices únicos de vuelta a códigos ISO
           factorize_result = pd.factorize(itp2022['importer_iso3'])
           codigos_countries = [factorize_result[1][idx] for idx in unique_indices]
       else:
           codigos_countries = list(itp2022['importer_iso3'].unique())
           
       print(f"Total de países únicos encontrados: {len(codigos_countries)}")

       return itp2022, codigos_countries

   except Exception as e:
       print(f"Error durante el procesamiento: {e}")
       raise

if __name__ == "__main__":
    try:
        data, countries = procesar_datos_itp()
        print("Procesamiento completado con éxito")
    except Exception as e:
        print(f"Error en la ejecución principal: {e}")

GPU disponible: NVIDIA GeForce RTX 4060 Laptop GPU
Directorio fuente: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\raw\ITP\ITPD_E_R03
Directorio destino: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed
Combinando archivos comprimidos...


Procesando archivos: 100%|██████████| 10/10 [00:17<00:00,  1.72s/it]


Leyendo archivo CSV...
Usando Dask para procesamiento en paralelo
Filtrando datos de 2022...
Archivo temporal eliminado
Total de países únicos encontrados: 236
Procesamiento completado con éxito


In [16]:
import pandas as pd

# Asumiendo que tu DataFrame se llama 'df' y las columnas son 'codigo_industria' y 'nombre_industria'
# Primero, elimina los duplicados para quedarte solo con las combinaciones únicas
df_unico = data[['industry_id', 'industry_descr']].drop_duplicates()

# Convierte las dos columnas a un diccionario
diccionario_industrias = dict(zip(df_unico['industry_id'], df_unico['industry_descr']))

base_path = Path.cwd().parent.parent
target_directory = base_path / "data" / "processed" / "Dependencias consolidadas"
target_filename = "industrias_id_nombre.csv"
target_filepath = target_directory / target_filename

industrias = pd.DataFrame(diccionario_industrias.items(), columns=['industry_id', 'industry_descr'])
industrias.to_csv(target_filepath, sep=";", index=False)


# Matrices de Comercio Internacional: Creación Optimizada con Aceleración GPU

El siguiente código implementa una función optimizada para crear matrices de comercio bilateral para cada industria a partir de datos comerciales internacionales. Esta implementación utiliza técnicas avanzadas para mejorar significativamente el rendimiento:

### Características principales:

- **Aceleración por GPU**: Detecta automáticamente si hay una GPU disponible y la utiliza para acelerar el procesamiento cuando hay suficientes datos.
- **Operaciones vectorizadas**: Minimiza las iteraciones fila por fila usando enfoques vectorizados más eficientes.
- **Gestión de memoria optimizada**: Reutiliza estructuras para reducir la fragmentación y el consumo de memoria.
- **Adaptabilidad**: Ajusta automáticamente la estrategia de procesamiento según el volumen de datos y el hardware disponible.

El resultado es un diccionario donde cada clave representa una industria, y cada valor es una matriz completa de comercio bilateral entre todos los países del conjunto de datos, lo que facilita los análisis posteriores de dependencias comerciales y flujos económicos.

Este enfoque es particularmente útil para el cálculo de índices de dependencia comercial, ya que proporciona una representación eficiente de los flujos comerciales entre países para cada sector industrial.

### Limpieza de Matrices de Comercio: Eliminación de Flujos No Significativos

El siguiente código implementa una función para eliminar del análisis aquellos países con flujos comerciales no significativos. Este paso es crucial para mejorar la precisión del análisis de dependencias comerciales, ya que permite enfocarse en relaciones económicas realmente relevantes.

**Funcionamiento**:

- **Umbral de significancia**: Establece un umbral mínimo (por defecto 0.05%) del comercio mundial total. Cualquier flujo comercial por debajo de este umbral se considera no significativo y se convierte a cero.
- **Identificación de países no relevantes**: Detecta países que, después de aplicar el umbral, no tienen flujos comerciales significativos (tanto como exportadores como importadores).
- **Limpieza de la matriz**: Elimina estos países de la matriz, reduciendo su dimensionalidad y concentrando el análisis en actores relevantes del comercio internacional.

Esta función de preprocesamiento es especialmente importante para estudios de dependencia económica, ya que:
1. Reduce el ruido en los datos al eliminar relaciones comerciales marginales
2. Mejora la eficiencia computacional al trabajar con matrices más pequeñas
3. Permite concentrarse en dependencias significativas que podrían representar vulnerabilidades reales

El resultado es una matriz "limpia" que contiene únicamente los países y flujos comerciales relevantes para el análisis posterior.

In [17]:
def crear_matriz_comercio_optimizado(grouped_data, codigos_paises: List[str]) -> Dict[str, pd.DataFrame]:
    """
    Crea matrices de comercio bilateral para cada industria a partir de datos agrupados,
    aprovechando operaciones vectorizadas y aceleración por GPU cuando está disponible.

    Args:
        grouped_data: DataFrame agrupado por industria.
        codigos_paises: Lista de códigos de países ISO3.

    Returns:
        Dict[str, pd.DataFrame]: Diccionario de matrices de comercio.
    """
    matrices = {}
    required_columns = {'exporter_iso3', 'importer_iso3', 'trade'}
    
    # Para el subproducto: crear una lista para almacenar los datos de totales
    totales_data = []
    
    # Verificar columnas requeridas
    if not required_columns.issubset(grouped_data.obj.columns):
        raise ValueError(f"Los datos deben contener las columnas: {required_columns}")
    
    # Crear un mapeo de códigos de países a índices para operaciones más rápidas
    pais_a_indice = {pais: i for i, pais in enumerate(codigos_paises)}
    
    # Verificar si hay GPU disponible
    use_gpu = torch.cuda.is_available()
    device = torch.device('cuda' if use_gpu else 'cpu')
    print(f"Usando {'GPU' if use_gpu else 'CPU'} para procesamiento")
    
    # Crear una matriz vacía template para reutilizar
    matriz_template = pd.DataFrame(
        0.0,
        index=codigos_paises,
        columns=codigos_paises
    )
    
    # Preprocesar datos para evitar iteraciones lentas
    for industry, group in tqdm(grouped_data, desc="Creando matrices de comercio"):
        # Filtrar solo las filas con países válidos
        valid_trades = group[
            group['exporter_iso3'].isin(codigos_paises) & 
            group['importer_iso3'].isin(codigos_paises)
        ]
        
        if valid_trades.empty:
            # Reutilizar la matriz template (hacer una copia para evitar modificar el original)
            matrices[industry] = matriz_template.copy()
            
            # Añadir entradas con valores cero para este caso
            for pais in codigos_paises:
                totales_data.append({
                    'pais': pais,
                    'industria': industry,
                    'valor_importado': 0.0
                })
            continue
        
        # Enfoque vectorizado para construir la matriz
        if use_gpu and len(valid_trades) > 1000:  # Solo usar GPU si hay suficientes datos
            # Convertir a índices numéricos para operaciones más rápidas
            exporters = valid_trades['exporter_iso3'].map(pais_a_indice).values
            importers = valid_trades['importer_iso3'].map(pais_a_indice).values
            values = valid_trades['trade'].values
            
            # Crear tensor en GPU
            matrix = torch.zeros((len(codigos_paises), len(codigos_paises)), device=device)
            
            # Usar scatter_add_ para construir la matriz eficientemente
            indices = torch.tensor(np.vstack([exporters, importers]), device=device, dtype=torch.long)
            matrix.index_put_(
                indices=(torch.tensor(exporters, device=device, dtype=torch.long),
                         torch.tensor(importers, device=device, dtype=torch.long)),
                values=torch.tensor(values, device=device, dtype=torch.float),
                accumulate=True
            )
            
            # Convertir de vuelta a DataFrame
            matrix_df = pd.DataFrame(
                matrix.cpu().numpy(),
                index=codigos_paises,
                columns=codigos_paises
            )
        else:
            # Crear una matriz completa de ceros y luego llenarla de una sola vez
            # Empezamos con una copia de la matriz template
            matrix_df = matriz_template.copy()
            
            # Preparar datos para actualización rápida
            for _, row in valid_trades.iterrows():
                exp = row['exporter_iso3']
                imp = row['importer_iso3']
                matrix_df.at[exp, imp] = row['trade']
                
        matrices[industry] = matrix_df
        
        # Calcular y guardar los totales importados por país para esta industria
        for pais in codigos_paises:
            # Sumar todos los valores de la columna (importaciones totales del país)
            valor_total = matrix_df[pais].sum()
            
            # Añadir a nuestros datos de totales
            totales_data.append({
                'pais': pais,
                'industria': industry,
                'valor_importado': valor_total
            })
        
    # Crear DataFrame con los totales
    df_totales = pd.DataFrame(totales_data)
    
    # Guardar en CSV
    target_filename = "totales_comercio_por_pais_industria.csv"
    target_filepath = target_directory / target_filename
    df_totales.to_csv(target_filepath, index=False, sep=";")
    print(f"Archivo CSV de totales guardado en: {target_filepath}")

    return matrices

# Define la lista de códigos de países
codigos_paises = sorted(data['importer_iso3'].unique().tolist())

# Llama a la función optimizada
matrices_comercio = crear_matriz_comercio_optimizado(data.groupby('industry_descr'), codigos_paises)

def eliminar_filas_columnas_cero(df, threshold_pct=0.005):
    """Filtra una matriz de comercio aplicando un umbral relativo a cada país importador."""
    # Crear una copia para no modificar el original
    df_filtered = df.copy()
    
    # Para cada país importador (columna), aplicar su propio umbral
    for country in df.columns:
        # Calcular el total de importaciones del país
        country_total_imports = df[country].sum()
        
        # Calcular el umbral específico para este país
        country_threshold = country_total_imports * (threshold_pct)
        
        # Convertir a cero los valores por debajo del umbral solo para este país
        df_filtered.loc[:, country] = df[country].where(df[country] >= country_threshold, 0)
    
    # Ahora identificar países con todos ceros
    zero_rows = df_filtered.index[df_filtered.sum(axis=1) == 0]
    zero_cols = df_filtered.columns[df_filtered.sum(axis=0) == 0]
    countries_to_drop = list(set(zero_rows) & set(zero_cols))
    
    # Eliminar los países identificados
    mat_clean = df_filtered.drop(countries_to_drop, axis=0).drop(countries_to_drop, axis=1)
    
    return mat_clean

Usando GPU para procesamiento


Creando matrices de comercio: 100%|██████████| 170/170 [00:04<00:00, 41.46it/s]

Archivo CSV de totales guardado en: c:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\processed\Dependencias consolidadas\totales_comercio_por_pais_industria.csv


# Cálculo de dependencias

La función `calculate_path_dependency` está diseñada para calcular la dependencia a lo largo de una cadena específica de nodos (países o entidades) en un análisis de flujos económicos.

Paso a paso:

1. **Entrada de la función**:
   - `X_clean`: Una matriz que representa los flujos entre nodos (países). Cada valor `X_clean[i, j]` representa el flujo desde el nodo `i` al nodo `j`.
   - `path`: Una lista que define un camino específico a través de varios nodos (países).
   - `denominators`: Un array que contiene los valores de normalización para cada nodo.

2. **Proceso**:
   - Inicia calculando el flujo directo entre los dos primeros nodos del camino: `flujo_actual = X_clean[path[0], path[1]]`.
   
   - Luego itera a través de cada nodo intermedio en el camino para calcular cómo el flujo se propaga:
     ```python
     for i in range(1, len(path)-1):
         nodo_actual = path[i]
     ```

   - Para cada nodo, verifica si puede haber flujo saliente (si el denominador es 0, no puede haber flujo):
     ```python
     if denominators[nodo_actual] == 0:
         return 0
     ```

   - Calcula qué proporción del flujo que llega al nodo actual continúa hacia el siguiente nodo:
     ```python
     proporcion = X_clean[path[i], path[i+1]] / denominators[nodo_actual]
     ```
     Esta proporción representa la fracción del flujo total saliente del nodo actual que va específicamente al siguiente nodo.

   - Multiplica el flujo actual por esta proporción para actualizar el valor:
     ```python
     flujo_actual = flujo_actual * proporcion
     ```
     Este paso acumula el efecto de cada transición a lo largo del camino.

3. **Resultado**:
   - Finalmente, normaliza el flujo resultante por el denominador del nodo final:
     ```python
     return flujo_actual / denominators[path[-1]] if denominators[path[-1]] > 0 else 0
     ```
     Esto garantiza que la dependencia se exprese como una proporción relativa al flujo total del nodo destino.

En resumen, esta función calcula cuánto de un flujo inicial sobrevive a través de toda una cadena de nodos, considerando en cada paso qué proporción del flujo continúa al siguiente nodo. Es útil para analizar dependencias en cadenas de suministro, flujos comerciales o relaciones económicas donde los recursos o bienes fluyen a través de múltiples intermediarios.

La función `calculate_path_dependency` puede expresarse formalmente usando la siguiente formulación en LaTeX:

$$\text{Dependencia}(p) = \frac{f_{p_1, p_2}}{d_{p_n}} \prod_{i=2}^{n-1} \frac{f_{p_i, p_{i+1}}}{d_{p_i}}$$

Donde:
- $p = [p_1, p_2, \ldots, p_n]$ es el camino (path) que consiste en $n$ nodos
- $f_{i,j}$ representa el flujo desde el nodo $i$ al nodo $j$ (corresponde a `X_clean[i, j]`)
- $d_i$ es el denominador para el nodo $i$ (el valor de normalización, típicamente la suma de todos los flujos salientes)

Esta fórmula se puede interpretar de la siguiente manera:
1. Comenzamos con el flujo inicial $f_{p_1, p_2}$ del primer nodo al segundo nodo
2. Para cada nodo intermedio $p_i$ (donde $2 \leq i \leq n-1$), calculamos la proporción del flujo que continúa al siguiente nodo $p_{i+1}$, expresado como $\frac{f_{p_i, p_{i+1}}}{d_{p_i}}$
3. Multiplicamos todas estas proporciones junto con el flujo inicial
4. Finalmente, normalizamos por el denominador del nodo final $d_{p_n}$

Si cualquiera de los denominadores $d_{p_i}$ es cero, entonces la dependencia completa se define como cero, ya que no puede haber flujo a través de ese nodo.

Formalmente, la condición completa se puede expresar como:

$$\text{Dependencia}(p) = 
\begin{cases} 
0 & \text{si } \exists i \in \{2, 3, \ldots, n\} : d_{p_i} = 0 \\
\frac{f_{p_1, p_2}}{d_{p_n}} \prod_{i=2}^{n-1} \frac{f_{p_i, p_{i+1}}}{d_{p_i}} & \text{en otro caso}
\end{cases}$$

Esta formulación captura la naturaleza secuencial de la propagación del flujo a lo largo de todo el camino, considerando las proporciones de transición en cada etapa.

## Ejemplo sencillo de dependencia de camino

Imaginemos un sistema con 4 países (A, B, C, D) con los siguientes flujos comerciales entre ellos:

- De A a B: 100 unidades
- De B a C: 50 unidades
- De C a D: 30 unidades
- De B a otros destinos: 150 unidades (total saliente de B = 200)
- De C a otros destinos: 70 unidades (total saliente de C = 100)
- Flujo total que recibe D: 80 unidades

Queremos calcular la dependencia a lo largo del camino p = [A, B, C, D].

### Paso 1: Definimos nuestros datos
- Matriz de flujos `X_clean`:
  ```
  X_clean = [
    [0, 100, 0, 0],  # Flujos desde A
    [0, 0, 50, 0],   # Flujos desde B
    [0, 0, 0, 30],   # Flujos desde C
    [0, 0, 0, 0]     # Flujos desde D
  ]
  ```

- Denominadores (totales de flujos salientes/entrantes según corresponda):
  ```
  denominators = [100, 200, 100, 80]
  ```

- Camino a analizar:
  ```
  path = [0, 1, 2, 3]  # Corresponde a A→B→C→D
  ```

### Paso 2: Calculamos la dependencia

1. Flujo inicial: flujo_actual = X_clean[0, 1] = 100

2. Para el nodo B (índice 1):
   - Proporción que va a C = X_clean[1, 2] / denominators[1] = 50/200 = 0.25
   - flujo_actual = 100 * 0.25 = 25

3. Para el nodo C (índice 2):
   - Proporción que va a D = X_clean[2, 3] / denominators[2] = 30/100 = 0.3
   - flujo_actual = 25 * 0.3 = 7.5

4. Normalización final:
   - Dependencia = flujo_actual / denominators[3] = 7.5/80 = 0.09375

### Interpretación

La dependencia de 0.09375 (o 9.375%) indica que este camino específico A→B→C→D representa aproximadamente el 9.4% del flujo total que recibe D.

En términos prácticos, esto podría significar que:
- El 9.4% de los bienes que llegan a D provienen originalmente de A, pasando por B y C.
- Si hubiera una interrupción en cualquier punto de esta cadena, aproximadamente el 9.4% de las importaciones de D estarían en riesgo.

Este cálculo captura la "dilución" del flujo original a medida que pasa por múltiples intermediarios, donde en cada paso solo una fracción continúa por el camino específico que estamos analizando.

In [18]:
def calculate_path_dependency(X_clean, path, denominators):
    """Calcula la dependencia de un camino específico."""
    fuerza_camino = 1.0
    for k in range(len(path) - 1):
        if denominators[path[k+1]] > 0:
            fuerza_camino *= X_clean[path[k], path[k+1]] / denominators[path[k+1]]
        else:
            fuerza_camino = 0  # Si el denominador es cero, la fuerza del camino es cero
    return fuerza_camino

def calculate_intermediary_centrality(intermediary_frequency, intermediary_strength, country_names):
    """
    Calcula métricas de centralidad para intermediarios.
    
    Esta función es la misma que la original.
    """
    # Implementación existente
    centrality = []
    
    # Normalizar
    max_freq = max(intermediary_frequency.values()) if intermediary_frequency.values() else 1
    max_strength = max(intermediary_strength.values()) if intermediary_strength.values() else 1
    
    for country in country_names:
        norm_freq = intermediary_frequency[country] / max_freq if max_freq > 0 else 0
        norm_strength = intermediary_strength[country] / max_strength if max_strength > 0 else 0
        
        combined_score = 0.4 * norm_freq + 0.6 * norm_strength
        
        centrality.append((country, intermediary_frequency[country], 
                          intermediary_strength[country], combined_score))
    
    centrality.sort(key=lambda x: x[3], reverse=True)
    return centrality

def process_country_pair(i, j, X_clean, denominators, country_names, max_possible_length, 
                        convergence_threshold, path_strength_threshold):
    """
    Procesa un par de países para calcular sus dependencias.
    Esta función será ejecutada en paralelo.
    """
    # Inicialización
    current_total = 0
    indirect_total = 0
    dependencies_by_length = {}
    significant_paths = []
    length = 1
    
    # Dependencia directa
    trade_value = X_clean[j, i]
    direct_dependency = X_clean[j, i] / denominators[i] if denominators[i] > 0 else 0
    dependencies_by_length[1] = direct_dependency
    current_total = direct_dependency
    
    # Clave para identificar el par de países
    pair_key = f"{country_names[j]}->{country_names[i]}"
    
    # Dependencias indirectas
    for length in range(2, max_possible_length + 1):
        DI_ij_l = 0
        middle_countries = [k for k in range(X_clean.shape[0]) if k != i and k != j]

        # Examinar cada combinación posible de intermediarios
        for intermediaries in combinations(middle_countries, length - 1):
            path = (j,) + intermediaries + (i,)
            fuerza_camino = calculate_path_dependency(X_clean, path, denominators)
            DI_ij_l += fuerza_camino
            
            # Registrar caminos significativos
            if fuerza_camino > path_strength_threshold:
                # Convertir índices a nombres de países
                path_countries = [country_names[idx] for idx in path]
                
                # Registrar este camino
                path_info = {
                    'exportador': path_countries[0],
                    'importador': path_countries[-1],
                    'intermediarios': path_countries[1:-1],
                    'fuerza': fuerza_camino,
                    'longitud': length
                }
                
                significant_paths.append(path_info)
                
                # No podemos actualizar directamente las estructuras globales en paralelo
                # Devolveremos estos valores para actualizar después
                intermediary_updates = []
                for interm_idx, interm_country in enumerate(path_countries[1:-1]):
                    weight_factor = 1.0 / (interm_idx + 1)
                    intermediary_updates.append((interm_country, weight_factor * fuerza_camino))
                
        dependencies_by_length[length] = DI_ij_l
        indirect_total += DI_ij_l

        prev_total = current_total
        current_total = direct_dependency + indirect_total

        # Criterio de convergencia
        if length > 1 and abs(current_total - prev_total) < convergence_threshold:
            break

    # La dependencia total es la suma de la directa y la indirecta
    total_dependency = direct_dependency + indirect_total

    result = {
        'importador': country_names[i],
        'exportador': country_names[j],
        'trade_value': trade_value,
        'dependencia_directa': direct_dependency,
        'dependencia_indirecta': indirect_total,
        'dependencia_total': total_dependency,
        'longitud_optima': length,
        'dependencias_por_longitud': dependencies_by_length
    }
    
    # Ordenar caminos significativos
    significant_paths.sort(key=lambda x: x['fuerza'], reverse=True)
    
    # Resultados a devolver para este par de países
    return {
        'pair_key': pair_key,
        'result': result,
        'top_dependency': (country_names[i], country_names[j], direct_dependency,
                         indirect_total, total_dependency, length),
        'significant_paths': significant_paths,
        'length_converged': length if length > 1 else 0
    }

def calculate_all_dependencies_parallel(X, country_names=None, convergence_threshold=0.01, 
                                       max_possible_length=3, 
                                       path_strength_threshold=0.001, n_jobs=None, use_gpu=True, 
                                       debug_mode=False):
    """
    Versión paralelizada del cálculo de dependencias que mantiene EXACTAMENTE
    la misma salida que la versión original.
    
    El parámetro debug_mode permite verificar que el número de dependencias
    coincida con la versión original.
    """
    """
    Versión paralelizada del cálculo de dependencias.
    
    Parameters adicionales:
    -----------------------
    n_jobs : int, opcional
        Número de trabajos paralelos. Si es None, usa todos los núcleos disponibles.
    use_gpu : bool, default=True
        Si se debe intentar usar GPU para acelerar algunos cálculos.
    """
    n = X.shape[0]

    if country_names is None:
        country_names = [f"País {i}" for i in range(n)]

    if len(country_names) != n:
        raise ValueError(f"La longitud de country_names ({len(country_names)}) no coincide con la dimensión de X ({n})")

    # Configurar paralelización
    if n_jobs is None:
        n_jobs = multiprocessing.cpu_count()
    
    # Verificar disponibilidad de GPU
    gpu_available = torch.cuda.is_available() and use_gpu
    X_clean = X

    denominators = np.sum(X, axis=0)

    # Acelerar cálculos directos con GPU si está disponible
    if gpu_available:
        # Transferir datos a GPU
        X_gpu = torch.tensor(X_clean, device='cuda', dtype=torch.float32)
        denom_gpu = torch.tensor(denominators, device='cuda', dtype=torch.float32)
        
        # Calcular dependencias directas en forma vectorizada
        direct_deps = torch.zeros_like(X_gpu)
        for i in range(n):
            # Evitar división por cero
            if denom_gpu[i] > 0:
                direct_deps[:, i] = X_gpu[:, i] / denom_gpu[i]
        
        # Transferir resultados de vuelta a CPU
        direct_dependencies = direct_deps.cpu().numpy()
        
        # Usar estas dependencias directas precalculadas en el procesamiento posterior
        # (Aunque en esta implementación seguimos calculándolas en process_country_pair para
        # mantener cambios mínimos en el código)

    # Estructura de resultados extendida
    results = {
        'dependencies': [],
        'top_dependencies': [],
        'avg_dependencies': {},
        'length_distribution': np.zeros(max_possible_length),
        'critical_intermediaries': {},     # Intermediarios críticos por relación
        'intermediary_frequency': {},      # Frecuencia de países como intermediarios
        'critical_paths': [],              # Rutas críticas completas
        'intermediary_strength': {}        # Fuerza de cada país como intermediario
    }
    
    # Inicializar contadores para intermediarios
    for country in country_names:
        results['intermediary_frequency'][country] = 0
        results['intermediary_strength'][country] = 0.0

    # Preparar pares de países para procesamiento paralelo 
    # Mantenemos la misma estructura de iteración del código original
    # Primero por importador (i) y luego por exportador (j)
    country_pairs = []
    for i in range(n):
        for j in range(n):
            if i != j:
                country_pairs.append((i, j))
    
    # Procesar pares de países en paralelo
    with Parallel(n_jobs=n_jobs) as parallel:
        pair_results = parallel(
            delayed(process_country_pair)(
                i, j, X_clean, denominators, country_names, 
                max_possible_length, convergence_threshold, path_strength_threshold
            ) 
            for i, j in tqdm(country_pairs, desc="Calculando dependencias")
        )
        
    # Agrupar resultados por país importador
    results_by_importer = {}
    for res in pair_results:
        importer = res['result']['importador']
        if importer not in results_by_importer:
            results_by_importer[importer] = []
        results_by_importer[importer].append(res)
    
    # Recolectar critical paths de todos los pares para ordenarlos después
    all_critical_paths = []
    
    # Procesar los resultados manteniendo el mismo orden que el código original
    for i in range(n):
        importer = country_names[i]
        total_dep = 0.0
        num_deps = 0
        
        if importer in results_by_importer:
            for res in results_by_importer[importer]:
                # Agregar a dependencies
                results['dependencies'].append(res['result'])
                
                # Agregar a top_dependencies
                results['top_dependencies'].append(res['top_dependency'])
                
                # Actualizar critical_intermediaries
                results['critical_intermediaries'][res['pair_key']] = res['significant_paths']
                
                # Recolectar critical paths
                all_critical_paths.extend(res['significant_paths'])
                
                # Actualizar length_distribution si convergió
                if res['length_converged'] > 1:
                    results['length_distribution'][res['length_converged'] - 1] += 1
                
                # Actualizar dependencia promedio
                total_dep += res['result']['dependencia_total']
                num_deps += 1
                
                # Actualizar estadísticas de intermediarios
                for path in res['significant_paths']:
                    for idx, interm in enumerate(path['intermediarios']):
                        # Incrementar frecuencia
                        results['intermediary_frequency'][interm] += 1
                        
                        # Incrementar fuerza ponderada
                        weight_factor = 1.0 / (idx + 1)
                        results['intermediary_strength'][interm] += path['fuerza'] * weight_factor
        
        # Guardar dependencia promedio para este importador
        results['avg_dependencies'][importer] = total_dep / num_deps if num_deps > 0 else 0
    
    # Añadir y ordenar los critical paths (igual que el original)
    results['critical_paths'] = all_critical_paths
    
    # Ordenar top dependencies
    results['top_dependencies'].sort(key=lambda x: x[4], reverse=True)
    results['top_dependencies'] = results['top_dependencies'][:90]
    
    # Ordenar critical_paths por fuerza
    results['critical_paths'].sort(key=lambda x: x['fuerza'], reverse=True)
    
    # Calcular métricas de centralidad para intermediarios
    results['intermediary_centrality'] = calculate_intermediary_centrality(
        results['intermediary_frequency'], 
        results['intermediary_strength'],
        country_names
    )
    
    return results

# Para mantener compatibilidad, redefinimos la función original
# para que utilice la versión paralelizada
def calculate_all_dependencies(X, country_names=None, convergence_threshold=0.01, 
                              max_possible_length=3, threshold_pct=0.01, 
                              path_strength_threshold=0.001):
    """
    Calcula todas las dependencias entre países con análisis de intermediarios críticos.
    
    Esta función mantiene EXACTAMENTE la misma firma y resultados que la original,
    pero utiliza internamente paralelización y GPU para acelerar los cálculos.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Matriz de comercio
    country_names : list, opcional
        Nombres de los países
    convergence_threshold : float, default=0.01
        Umbral para determinar la convergencia
    max_possible_length : int, default=5
        Longitud máxima de caminos a considerar
    threshold_pct : float, default=0.01
        Umbral para filtrar valores de comercio insignificantes (porcentaje)
    path_strength_threshold : float, default=0.001
        Umbral mínimo para considerar una ruta como significativa
        
    Returns:
    --------
    dict
        Diccionario con todos los resultados del análisis
    """
    # Determinar si usar paralelización basado en el tamaño del problema
    use_parallel = X.shape[0] > 5  # Para matrices muy pequeñas no vale la pena paralelizar
    
    # Verificar disponibilidad de GPU
    use_gpu = torch.cuda.is_available()
    
    # Aquí agregas el segundo log
    print(f"Usando paralelización: {use_parallel}, GPU disponible: {use_gpu}")
    if use_gpu:
        print(f"GPU en uso: {torch.cuda.get_device_name(0)}")
    
    # Configurar número de trabajos para paralelización
    n_countries = X.shape[0]
    n_jobs = min(multiprocessing.cpu_count(), n_countries)  # Limitar al número de países
    
    
    if use_parallel:
        # Usar la versión paralelizada
        return calculate_all_dependencies_parallel(
            X, country_names, convergence_threshold, 
            max_possible_length, path_strength_threshold,
            n_jobs=n_jobs, use_gpu=use_gpu, debug_mode=False
        )
    else:
        # Para matrices muy pequeñas, usar un solo proceso
        return calculate_all_dependencies_parallel(
            X, country_names, convergence_threshold, 
            max_possible_length, path_strength_threshold,
            n_jobs=1, use_gpu=use_gpu, debug_mode=False
        )


## Generar y guardar resultados de dependencias

In [19]:

# Diccionario para guardar todos los resultados
all_results = {}

# Procesar cada industria
for industry, mat in matrices_comercio.items():
    print(f"\nProcesando industria: {industry}")
    
    # Limpiar matriz
    mat_clean = eliminar_filas_columnas_cero(mat)
    
    # Si la matriz limpia está vacía o es muy pequeña, continuamos con la siguiente
    if mat_clean.shape[0] < 2:
        print(f"Matriz demasiado pequeña para industria {industry}")
        continue
        
    # Convertir a numpy array y obtener nombres de países
    X = mat_clean.values
    country_names = mat_clean.columns.tolist()

    # Aquí agregas solo el log del tamaño de la matriz
    print(f"Matriz para {industry}: {mat_clean.shape}")
    
    # Verificar disponibilidad de GPU (solo para información)
    gpu_available = torch.cuda.is_available()
    print(f"GPU disponible: {gpu_available}")
    if gpu_available:
        print(f"GPU en uso: {torch.cuda.get_device_name(0)}")
     
    # Calcular dependencias
    results = calculate_all_dependencies(X, country_names)
    
    # Guardar resultados
    all_results[industry] = {
        'results': results,
        'country_names': country_names,
        'matrix_shape': mat_clean.shape
    }


Procesando industria: 154 Manufacturing services on physical inputs
Matriz para 154 Manufacturing services on physical inputs: (64, 64)
GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU
Usando paralelización: True, GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU


Calculando dependencias: 100%|██████████| 4032/4032 [00:02<00:00, 1948.16it/s]



Procesando industria: 155 Maintenance and repair services n.i.e.
Matriz para 155 Maintenance and repair services n.i.e.: (78, 78)
GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU
Usando paralelización: True, GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU


Calculando dependencias: 100%|██████████| 6006/6006 [00:03<00:00, 1897.48it/s]



Procesando industria: 156 Transport
Matriz para 156 Transport: (98, 98)
GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU
Usando paralelización: True, GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU


...

Calculando dependencias: 100%|██████████| 32220/32220 [00:24<00:00, 1305.08it/s]



Procesando industria: Wines
Matriz para Wines: (231, 231)
GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU
Usando paralelización: True, GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU


Calculando dependencias: 100%|██████████| 53130/53130 [01:18<00:00, 680.52it/s] 



Procesando industria: Wooden containers
Matriz para Wooden containers: (219, 219)
GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU
Usando paralelización: True, GPU disponible: True
GPU en uso: NVIDIA GeForce RTX 4060 Laptop GPU


Calculando dependencias: 100%|██████████| 47742/47742 [01:23<00:00, 572.19it/s]


## Generar y guadar datos de intermediarios Críticos

In [20]:
def get_top_intermediaries(results, top_n=10):
    """
    Obtiene los países más importantes como intermediarios.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis con intermediarios
    top_n : int
        Número de países a mostrar
        
    Returns:
    --------
    list
        Lista de tuplas (país, score, frecuencia, fuerza) ordenadas por importancia
    """
    # La estructura de intermediary_centrality ha cambiado en la nueva implementación
    # Ahora es una lista de tuplas (país, frecuencia, fuerza, score_combinado)
    if isinstance(results['intermediary_centrality'], list):
        # Nueva implementación: ya está ordenada por score combinado
        return results['intermediary_centrality'][:top_n]
    else:
        # Implementación anterior (por compatibilidad)
        centrality_scores = [(country, metrics) 
                            for country, metrics in results['intermediary_centrality'].items()]
        centrality_scores.sort(key=lambda x: x[1]['centrality_score'], reverse=True)
        return centrality_scores[:top_n]

def analyze_country_intermediary_role(results, country):
    """
    Analiza el papel de un país específico como intermediario.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis con intermediarios
    country : str
        País a analizar
        
    Returns:
    --------
    dict
        Información detallada sobre el rol del país como intermediario
    """
    # Verificar si el país existe en los resultados
    if isinstance(results['intermediary_centrality'], list):
        # Nueva implementación - lista de tuplas
        country_entry = next((entry for entry in results['intermediary_centrality'] 
                             if entry[0] == country), None)
        
        if country_entry is None:
            return {"error": f"El país {country} no está en los resultados"}
            
        # Extraer métricas
        centrality = {
            "frequency": country_entry[1],
            "strength": country_entry[2],
            "centrality_score": country_entry[3]
        }
    else:
        # Implementación anterior
        if country not in results['intermediary_centrality']:
            return {"error": f"El país {country} no está en los resultados"}
        centrality = results['intermediary_centrality'][country]
    
    # Encontrar las rutas más importantes donde este país actúa como intermediario
    top_paths = []
    for path in results['critical_paths']:
        if country in path['intermediarios']:
            top_paths.append(path)
    
    # Ordenar por fuerza del camino
    top_paths.sort(key=lambda x: x['fuerza'], reverse=True)
    
    # Filtrar los 10 caminos más importantes
    top_paths = top_paths[:10]
    
    return {
        "metrics": centrality,
        "top_paths": top_paths,
        "role_summary": {
            "total_paths": len(top_paths),
            "average_path_strength": sum(p['fuerza'] for p in top_paths) / len(top_paths) if top_paths else 0,
            "max_path_strength": max(p['fuerza'] for p in top_paths) if top_paths else 0,
            "unique_exporters": len(set(p['exportador'] for p in top_paths)),
            "unique_importers": len(set(p['importador'] for p in top_paths))
        }
    }

def summarize_country_dependencies(results, country, top_n=5):
    """
    Genera un resumen de las dependencias comerciales de un país específico.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis de dependencias
    country : str
        País a analizar
    top_n : int
        Número de dependencias principales a mostrar
        
    Returns:
    --------
    dict
        Resumen de dependencias del país
    """
    # Dependencias como importador (otros países exportan a este país)
    import_dependencies = []
    for dep in results['dependencies']:
        if dep['importador'] == country:
            import_dependencies.append(dep)
    
    # Ordenar por dependencia total
    import_dependencies.sort(key=lambda x: x['dependencia_total'], reverse=True)
    
    # Dependencias como exportador (este país exporta a otros)
    export_dependencies = []
    for dep in results['dependencies']:
        if dep['exportador'] == country:
            export_dependencies.append(dep)
    
    # Ordenar por dependencia total
    export_dependencies.sort(key=lambda x: x['dependencia_total'], reverse=True)
    
    # Calcular dependencia promedio
    avg_dependency = results['avg_dependencies'].get(country, 0)
    
    # Analizar el papel como intermediario
    intermediary_role = None
    if isinstance(results['intermediary_centrality'], list):
        # Nueva implementación
        intermediary_info = next((x for x in results['intermediary_centrality'] if x[0] == country), None)
        if intermediary_info:
            intermediary_role = {
                "frequency": intermediary_info[1],
                "strength": intermediary_info[2],
                "centrality_score": intermediary_info[3],
                "rank": next((i+1 for i, x in enumerate(results['intermediary_centrality']) 
                             if x[0] == country), None)
            }
    else:
        # Implementación anterior
        if country in results['intermediary_centrality']:
            intermediary_role = results['intermediary_centrality'][country]
            # Calcular rango
            countries_sorted = sorted(results['intermediary_centrality'].keys(), 
                                     key=lambda x: results['intermediary_centrality'][x]['centrality_score'],
                                     reverse=True)
            intermediary_role["rank"] = countries_sorted.index(country) + 1
    
    return {
        "country": country,
        "avg_dependency": avg_dependency,
        "top_import_dependencies": import_dependencies[:top_n],
        "top_export_dependencies": export_dependencies[:top_n],
        "total_import_dependencies": len(import_dependencies),
        "total_export_dependencies": len(export_dependencies),
        "intermediary_role": intermediary_role
    }

def identify_critical_trade_relationships(results, threshold=0.7, min_paths=3):
    """
    Identifica relaciones comerciales críticas con alta dependencia.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis de dependencias
    threshold : float
        Umbral de dependencia para considerar una relación como crítica
    min_paths : int
        Número mínimo de caminos alternativos para evitar criticidad
        
    Returns:
    --------
    list
        Lista de relaciones críticas
    """
    critical_relationships = []
    
    # Analizar cada dependencia
    for dep in results['dependencies']:
        if dep['dependencia_total'] >= threshold:
            # Buscar caminos alternativos
            pair_key = f"{dep['exportador']}->{dep['importador']}"
            alternative_paths = []
            
            if pair_key in results['critical_intermediaries']:
                alternative_paths = results['critical_intermediaries'][pair_key]
            
            # Si hay pocos caminos alternativos, es una relación crítica
            if len(alternative_paths) < min_paths:
                critical_relationships.append({
                    "exportador": dep['exportador'],
                    "importador": dep['importador'],
                    "dependencia_total": dep['dependencia_total'],
                    "dependencia_directa": dep['dependencia_directa'],
                    "caminos_alternativos": len(alternative_paths),
                    "criticidad": 1.0 - (len(alternative_paths) / min_paths) 
                                     if min_paths > 0 else 1.0
                })
    
    # Ordenar por criticidad
    critical_relationships.sort(key=lambda x: x['criticidad'], reverse=True)
    
    return critical_relationships

In [21]:
interm = get_top_intermediaries(all_results['Aircraft and spacecraft']['results'])
role = analyze_country_intermediary_role(results, 'DEU')

## Creo Dataframes para guardar resultados

In [ ]:
import pandas as pd

def create_dependencies_dataframe(all_results):
    """
    Crea un DataFrame con los resultados de dependencias para todas las industrias.
    
    Parameters:
    -----------
    all_results : dict
        Diccionario con los resultados por industria
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame con las columnas: industria, importador, exportador, 
        dependencia_total, dependencia_directa, dependencia_indirecta, longitud_optima
    """
    # Lista para almacenar los datos de todas las industrias
    all_data = []
    
    # Procesar cada industria
    for industry, data in all_results.items():
        # Obtener los resultados de esta industria
        industry_results = data['results']['dependencies']
        
        # Añadir cada fila de resultados
        for result in industry_results:
            row = {
                'industria': industry,
                'importador': result['importador'],
                'exportador': result['exportador'],
                'dependencia_total': result['dependencia_total'],
                'dependencia_directa': result['dependencia_directa'],
                'dependencia_indirecta': result['dependencia_indirecta'],
                'trade_value': result['trade_value'],
                'longitud_optima': result['longitud_optima']
            }
            all_data.append(row)
    
    # Crear el DataFrame
    df = pd.DataFrame(all_data)
    
    # Ordenar el DataFrame
    df = df.sort_values(['industria', 'dependencia_total'], ascending=[True, False])
    
    return df

# Ejemplo de uso:
df = create_dependencies_dataframe(all_results)

# Definir el nuevo mapeo de nombres de columnas
nuevo_nombres = {
    'industria': 'industry',
    'importador': 'dependent_country',
    'exportador': 'supplier_country',
    'dependencia_total': 'dependency_value',
    'dependencia_directa': 'direct_dependency',
    'dependencia_indirecta': 'indirect_dependency',
    'trade_value': 'trade_value',
    'longitud_optima': 'longitud_optima'
}

# Renombrar las columnas
df = df.rename(columns=nuevo_nombres)

# Guardar como gzip
with gzip.open(r"C:\Users\Usuario\Documents\Github\Seguridad Economica\src\data\processed\Dependencias consolidadas\dependencias.csv.gz", 'wt') as f:
    df.to_csv(f, sep=";", index=False)

# Lo que propone CHATGPT para hacer la seccion 6 del paper

In [41]:
import os
import pandas as pd
import gzip
from pathlib import Path


current_dir = Path.cwd().parent.parent
output_dir = current_dir / "data" / "processed" / "ficheros_paper"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------------------------
# 1. Dependencias completas (industria-país-par)
df = create_dependencies_dataframe(all_results)

# Renombrado (ya implementado antes)
df = df.rename(columns={
    'industria': 'industry',
    'importador': 'dependent_country',
    'exportador': 'supplier_country',
    'dependencia_total': 'dependency_value',
    'dependencia_directa': 'direct_dependency',
    'dependencia_indirecta': 'indirect_dependency',
    'trade_value': 'trade_value',
    'longitud_optima': 'optimal_length'
})

# Guardar
df.to_csv(os.path.join(output_dir, "dependencies_full.csv.gz"), sep=";", index=False, compression="gzip")

# -----------------------------------------------
# 2. Dependencias ponderadas bilateralmente
def safe_weighted_average(group):
    if group['trade_value'].sum() == 0:
        return np.nan
    return np.average(group['dependency_value'], weights=group['trade_value'])

weighted_dependencies = df.groupby(['dependent_country', 'supplier_country']).apply(
    safe_weighted_average
).reset_index(name='weighted_dependency')

weighted_dependencies.dropna().to_csv(
    os.path.join(output_dir, "weighted_dependencies.csv.gz"), sep=";", index=False, compression="gzip"
)

# -----------------------------------------------
# 3. Relaciones críticas
critical_relationships = []

for industry, data in all_results.items():
    crits = identify_critical_trade_relationships(data['results'], threshold=0.7, min_paths=3)
    for c in crits:
        c['industry'] = industry
    critical_relationships.extend(crits)

df_critical = pd.DataFrame(critical_relationships)
df_critical.to_csv(os.path.join(output_dir, "critical_relations.csv.gz"), sep=";", index=False, compression="gzip")

# -----------------------------------------------
# 4. Intermediarios por industria
intermediary_centrality_all = []

for industry, data in all_results.items():
    for c in data['results']['intermediary_centrality']:
        intermediary_centrality_all.append({
            'industry': industry,
            'country': c[0],
            'frequency': c[1],
            'strength': c[2],
            'centrality_score': c[3]
        })

df_centrality = pd.DataFrame(intermediary_centrality_all)
df_centrality.to_csv(os.path.join(output_dir, "intermediary_roles.csv.gz"), sep=";", index=False, compression="gzip")

# -----------------------------------------------
# 5. Centralidad global agregada
global_centrality = df_centrality.groupby("country")[["frequency", "strength", "centrality_score"]].sum()
global_centrality["centrality_rank"] = global_centrality["centrality_score"].rank(ascending=False)
global_centrality = global_centrality.sort_values("centrality_score", ascending=False)
global_centrality.reset_index().to_csv(
    os.path.join(output_dir, "intermediary_summary.csv.gz"), sep=";", index=False, compression="gzip"
)

print("✅ Todos los archivos CSV han sido generados en:", output_dir)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_26692\135950425.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_dependencies = df.groupby(['dependent_country', 'supplier_country']).apply(


✅ Todos los archivos CSV han sido generados en: c:\Users\Usuario\Documents\Github\Seguridad económica\src\data\processed\ficheros_paper


Sección 6.3 - Agregados de dependencia ponderada

In [ ]:
# 1. Promedio ponderado de dependencia por industria
industry_avg = df.groupby("industry").apply(safe_weighted_average).reset_index(name="avg_weighted_dependency")

# 2. Top 10 dependencias ponderadas bilaterales
top_10_bilateral = weighted_dependencies.sort_values("weighted_dependency", ascending=False).head(10)

# 3. Clasificación por tramos de dependencia
df["dependency_level"] = pd.cut(
    df["dependency_value"],
    bins=[0, 0.3, 0.7, 0.9, 1],
    labels=["baja", "media", "alta", "crítica"]
)

dependency_levels_summary = df["dependency_level"].value_counts().reset_index(name="num_relaciones")


Sección 6.4 - Relaciones críticas con baja redundancia

In [ ]:
# Asumiendo que 'all_results' está disponible (output del cálculo de dependencias por industria)
from collections import defaultdict

# 1. Extraer relaciones críticas por industria
critical_relations_all = []

for industry, data in all_results.items():
    crits = identify_critical_trade_relationships(data['results'], threshold=0.7, min_paths=3)
    for c in crits:
        c["industry"] = industry
    critical_relations_all.extend(crits)

# 2. Crear dataframe consolidado
df_critical = pd.DataFrame(critical_relations_all)

# 3. Ranking de países más afectados como importadores
importadores_riesgo = df_critical.groupby("importador").size().reset_index(name="relaciones_criticas")

# 4. Ranking como exportadores
exportadores_riesgo = df_critical.groupby("exportador").size().reset_index(name="exportaciones_criticas")

# 5. Exportar si se desea
# df_critical.to_csv("relaciones_criticas.csv", index=False)


Sección 6.5 - Intermediarios críticos y centralidad

In [ ]:
# 1. Consolidar centralidad de intermediarios para todas las industrias
all_centrality = []

for industry, data in all_results.items():
    cent = data['results']['intermediary_centrality']
    for entry in cent:
        all_centrality.append({
            "industry": industry,
            "country": entry[0],
            "frequency": entry[1],
            "strength": entry[2],
            "centrality_score": entry[3]
        })

df_centrality = pd.DataFrame(all_centrality)

# 2. Top 10 intermediarios globales (acumulando por país)
global_centrality = df_centrality.groupby("country")[["frequency", "strength", "centrality_score"]].sum()
global_centrality["centrality_rank"] = global_centrality["centrality_score"].rank(ascending=False)
global_centrality = global_centrality.sort_values("centrality_score", ascending=False)

# 3. Exportar si se desea
# df_centrality.to_csv("centralidad_intermediarios_por_industria.csv", index=False)
# global_centrality.to_csv("intermediarios_globales.csv")


Visualizaciones sugeridas (iniciales en matplotlib/seaborn)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Heatmap: dependencia promedio por industria y país
pivot_df = df.pivot_table(index="dependent_country", columns="industry", values="dependency_value", aggfunc="mean")
plt.figure(figsize=(18, 10))
sns.heatmap(pivot_df, cmap="YlGnBu", center=0.5)
plt.title("Dependencia media por país e industria")
plt.tight_layout()
plt.show()

# Burbujas: Intermediarios
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=global_centrality.reset_index(),
    x="frequency", y="strength", size="centrality_score", hue="centrality_score", sizes=(100, 1000), palette="viridis"
)
plt.title("Intermediarios clave según frecuencia y fuerza")
plt.xlabel("Frecuencia como intermediario")
plt.ylabel("Fuerza acumulada")
plt.tight_layout()
plt.show()

# Dispersión: criticidad vs. caminos alternativos
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_critical, x="caminos_alternativos", y="dependencia_total", hue="criticidad", palette="coolwarm")
plt.title("Relaciones críticas: dependencia vs. redundancia")
plt.xlabel("Número de caminos alternativos")
plt.ylabel("Dependencia total")
plt.tight_layout()
plt.show()
